In [1]:
import numpy as np

In [46]:
latent_dir = '/data/scratch/bariskurtkaya/dataset/NIRCAM/1386/models/modelv1/latent_v1/374_latent_space.npy'

In [47]:
latent = np.load(latent_dir, allow_pickle=True)

In [48]:
import pandas as pd

encoded_samples = []

for idx in range(latent.shape[0]):
    vector = np.reshape(latent[idx][0], (-1, 1))
    encoded_sample = {f"Enc. Variable {i}": enc for i, enc in enumerate(vector)}
    encoded_sample['label'] = latent[idx][1].cpu().numpy()[0]
    encoded_samples.append(encoded_sample)

encoded_samples = pd.DataFrame(encoded_samples)

In [ ]:
from sklearn.manifold import TSNE
import plotly.express as px

tsne = TSNE(n_components=2)
tsne_results = tsne.fit_transform(encoded_samples.drop(['label'],axis=1))
fig = px.scatter(tsne_results, x=0, y=1,
                 color=encoded_samples.label.astype(str),
                 labels={'0': 'tsne-2d-one', '1': 'tsne-2d-two'})
fig.show()